In [4]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [5]:
def synthetic_data(w, b, num_examples): #@save
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(-1, 1)

In [7]:
 true_w = torch.tensor([2,-3.4])
 true_b = 4.2
 features, labels = synthetic_data(true_w, true_b, 1000)

In [16]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)

    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i+batch_size, num_examples)]
        )
        yield features[batch_indices], labels[batch_indices]

In [17]:
batch_size=10
for X,y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-0.8431,  0.0049],
        [ 0.1755, -0.6732],
        [ 0.1138,  0.9109],
        [-0.3674,  0.5456],
        [-0.4277, -1.0099],
        [ 0.8762,  0.4693],
        [ 1.9389,  1.6376],
        [-0.2844, -0.7785],
        [ 1.5063, -1.2187],
        [-1.3313, -0.6043]]) 
 tensor([[ 2.5061],
        [ 6.8283],
        [ 1.3392],
        [ 1.5984],
        [ 6.7864],
        [ 4.3545],
        [ 2.5303],
        [ 6.2752],
        [11.3550],
        [ 3.5874]])


In [18]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [19]:
def linreg(X, w, b):
    return torch.matmul(X, w)+b

In [20]:
def squared_loss(y_hat, y):
    return (y_hat-y.reshape(y_hat.shape))**2/2

In [21]:
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

In [22]:
lr=0.03
num_epochs = 3
net = linreg
loss = squared_loss

In [23]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch+1}, loss {float(train_l.mean()):f}')

epoch1, loss 0.036401
epoch2, loss 0.000128
epoch3, loss 0.000049


In [24]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([5.5432e-04, 2.4796e-05], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0005], grad_fn=<RsubBackward1>)
